In [4]:
import pandas as pd
import ipywidgets as widgets
import os

import sys
sys.path.append("..")
from blog_tools import interact
%matplotlib inline

In [19]:
import numpy as np
import time
from IPython.display import display, clear_output


data_type = 'paths'
image_path="../img/quant/" + data_type + "/{}.png"
data_path="../data/quant/" + data_type + "/{}.csv"
max_seed=100

np.random.seed(int(time.time()))
table_widget = widgets.Output()
average_table_widget = widgets.Output()
image_widget = widgets.Image()

def change_image(seed):
    with open(image_path.format(seed), 'rb') as handle:
        png = handle.read()
    image_widget.value = png

def change_table(seed):
    global average_csv
    global n_samples
    csv = pd.read_csv(data_path.format(seed), index_col=0)
    csv = csv.set_index('method').T.round(3)
    csv.index.name=''
    try:
        average_csv *= n_samples
        n_samples += 1
        average_csv += csv
        average_csv /= n_samples
    except NameError:
        average_csv = csv
        n_samples = 1
    order = average_csv.T.sort_values('DEMaP', ascending=False).index
    with table_widget:
        clear_output()
        display(csv[order])
    with average_table_widget:
        clear_output()
        display(average_csv[order].round(3))

def randomize(*args, max_failures=max_seed, **kwargs):
    for _ in range(max_failures):
        try:
            seed = np.random.choice(max_seed)
            change_table(seed)
            change_image(seed)
            return seed
        except FileNotFoundError as e:
            pass
    raise FileNotFoundError("no valid files found in {} or {}".format(image_path, data_path))

randomizer = widgets.Button(description='Randomize!', tooltip='Change the random seed for the simulation')
randomizer.on_click(randomize)
dash = widgets.VBox([randomizer, image_widget, table_widget, average_table_widget])
randomize()
display(dash)